In [ ]:
import numpy as numpy
import pandas as pd
import matplotlib as mpl
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
import joblib
from sklearn.svm import SVC
from scipy.stats import loguniform
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

train_df = pd.read_csv("combined_train.csv")
test_df = pd.read_csv("combined_test.csv")
train_metadata = pd.read_csv("train/train_metadata.csv")
test_metadata = pd.read_csv("test/test_metadata.csv")

class_ids = train_metadata["ClassId"]
test_ids = test_metadata['id']
combined_train = train_df.drop("image_path", axis = 1)
combined_test = test_df.drop("image_path", axis=1)

digits = pd.read_csv("./dig_feature/ocr_digit_detection.csv")
pixels = pd.read_csv("./image_flat/flattened_images.csv")
cnn = pd.read_csv("./CNN_feature/cnn_features.csv")

new_train = pd.concat([train_df, cnn, digits], axis=1, join="inner")
new_train = new_train.drop("image_path", axis=1)


n_estimators =  [200, 500, 1000, 2000]
max_features = [0.01, 0.1, 'sqrt', "log2"]
max_depth = list(range(20, 111, 10))
max_depth.append(None)
min_samples_split = [2, 3, 4, 5]
min_samples_leaf = [1, 2, 3, 0.5]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

model = RandomForestClassifier(class_weight='balanced', verbose=1)
rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 250, cv = 3, verbose=1, n_jobs = -1, error_score='raise')
rf_random.fit(new_train, class_ids)
joblib.dump(rf_random, "post-RF.pkl")

Fitting 3 folds for each of 250 candidates, totalling 750 fits
